In [1]:
import requests
import time

In [2]:
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
import sys
sys.executable

'/Users/sunwoo/opt/anaconda3/bin/python'

In [11]:
options = webdriver.ChromeOptions() 
options.headless = True # it's more scalable to work in headless mode 
# normally, selenium waits for all resources to download 
# we don't need it as the page also populated with the running javascript code. 
options.page_load_strategy = 'none' 
# this returns the path web driver downloaded 
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
# pass the defined options and service objects to initialize the web driver 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

/var/folders/32/4hkbkvvs1lg8nc4zqtqfdf1r0000gn/T/ipykernel_17505/3638504904.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True # it's more scalable to work in headless mode


In [5]:
teams = ['Man_City','Chelsea','Bayern','Inter','AC Milan','Napoli','Real Madrid','Benfica']
urls =[]
with open('team_url.txt') as f:
    lines = f.readlines()
    for l in lines:
        urls.append(l)

In [6]:
teams2 = ['Liverpool','Spurs','Brugge','Dourtmund','Frankfurt','Leipzig','Porto','PSG']
urls2 =[]
with open('team_url2.txt') as f:
    lines = f.readlines()
    for l in lines:
        urls2.append(l)

In [7]:
def playerNum(p):
    return p.get_attribute('onclick').split("'")[3]
def playerName(p):
    return p.find_element(By.CLASS_NAME,'name').get_attribute('innerHTML')
def playerOverall(p):
    temp = p.find_element(By.CLASS_NAME, "skillData_"+playerNum(p))
    return int(temp.get_attribute("innerHTML"))
def playerBoost(p):
    boost = 0
    for i in p.find_elements(By.TAG_NAME, "span"):
        if i.get_attribute('class') == 'live up':
            boost = int(i.find_element(By.TAG_NAME, "i").get_attribute('innerHTML'))
            break
        elif i.get_attribute('class') == 'live down':
            boost = int(i.find_element(By.TAG_NAME, "i").get_attribute('innerHTML'))
            break
    return boost

In [12]:
names=[]
ovrs=[]
boosts =[]
for team,url in zip(teams,urls):
    driver.get(url) 
    time.sleep(20)
    table = driver.find_element(By.ID,"divPlayerList")
    players = table.find_elements(By.CLASS_NAME, "tr")
    print(team)
    ns = []
    ovs = []
    bs=[]
    for i in players:
        ns.append(playerName(i))
        ovs.append(playerOverall(i))
        bs.append(playerBoost(i))
    names.append(ns)
    ovrs.append(ovs)
    boosts.append(bs)

Man_City
Chelsea
Bayern
Inter
AC Milan
Napoli
Real Madrid
Benfica


In [13]:
names2=[]
ovrs2=[]
boosts2 =[]
for url in urls2:
    driver.get(url) 
    time.sleep(20)
    table = driver.find_element(By.ID,"divPlayerList")
    players = table.find_elements(By.CLASS_NAME, "tr")
    ns = []
    ovs = []
    bs=[]
    for i in players:
        ns.append(playerName(i))
        ovs.append(playerOverall(i))
        bs.append(playerBoost(i))
    names2.append(ns)
    ovrs2.append(ovs)
    boosts2.append(bs)

In [14]:
datas=[]
for players,team,overalls,bsts in zip(names,teams,ovrs,boosts):
    for player,overall,boost in zip(players,overalls,bsts):
        temp = [player,team,overall,boost]
        datas.append(temp)
for players,team,overalls,bsts in zip(names2,teams,ovrs,boosts):
    for player,overall,boost in zip(players,overalls,bsts):
        temp = [player,team,overall,boost]
        datas.append(temp)

In [15]:
len(datas)

349

In [16]:
import pandas as pd

In [17]:
dataset = pd.DataFrame(datas)
dataset.columns = ['name','team','ovr','boost']
dataset['og_ovr'] = dataset['ovr']- dataset['boost']
dataset.head()

,name,team,ovr,boost,og_ovr
0,엘링 홀란,Man_City,110,1,109
1,후벵 디아스,Man_City,109,2,107
2,케빈 더브라위너,Man_City,108,1,107
3,리야드 마레즈,Man_City,108,0,108
4,로드리,Man_City,107,2,105


In [18]:
dataset.to_csv('players.csv')

In [62]:
driver.get("https://fbref.com/en/matches/9b5a5b22/Manchester-City-Bayern-Munich-April-11-2023-Champions-League")
time.sleep(20)

In [92]:
playerTable = driver.find_elements(By.CLASS_NAME,"switcher_content")
playerTable = playerTable[:2]

In [81]:
i = playerTable[0]
teamNum = i.get_attribute("id").split('_')[-1]
table = i.find_element(By.ID,"stats_"+teamNum+"_summary")

In [104]:
players = table.find_elements(By.XPATH,"//tr")

In [105]:
len(players)

326

In [110]:
cnt=0
ps = []
for i in players:
    num = i.get_attribute("data-row")
    if num and int(num)<12:
        ps.append(i)
        cnt+=1
        if cnt >= 12:
            break

In [93]:
int(table.find_element(By.XPATH,"//tfoot").find_element(By.CLASS_NAME,"left").get_attribute("innerHTML").split()[0])

12